In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, sum as spark_sum, count, when, isnan, isnull, length, regexp_extract
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("CSVAnalysis").getOrCreate()

# Raw CSV content as a single row
csv_content = "1,2,3,4,5,ABC,XYZ,6,7,ABC,DEF"
df = spark.createDataFrame([(csv_content,)], ["raw_data"])

# Split into individual values
df_values = df.select(explode(split(col("raw_data"), ",")).alias("value"))
df_values.show(truncate=False)


In [0]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, sum as _sum

# spark = SparkSession.builder.getOrCreate()

# # Read CSV (single column, no header)
# df = spark.read.option("header", "false").csv("/path/to/file.csv") \
#        .withColumnRenamed("_c0", "value")

# Try casting to int
df_cast = df_values.withColumn("int_value", col("value").cast("int"))

# --------------------
# DF 1: Sum of integers
# --------------------
sum_df = df_cast.filter(col("int_value").isNotNull()) \
                .agg(_sum("int_value").alias("sum_of_integers"))

# --------------------
# DF 2: Count of each word
# --------------------
word_count_df = df_cast.filter(col("int_value").isNull()) \
                       .groupBy("value") \
                       .count()

# Show results
sum_df.show()
word_count_df.show()
